In [ ]:
import requests
import pandas as pd
import glob
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm

# OCR PDF

In [ ]:
def ocr(pdf_file, pdf_path):
    apiurl = "x"
    payload = {'stringInput': 'ocr'}
    files=[
    ('file',(pdf_file, open(pdf_path,'rb'),'application/pdf'))
    ]
    headers = {
    'accept': 'application/json'
    }

    return requests.request("POST", apiurl, headers=headers, data=payload, files=files, timeout=360)


In [ ]:
result = ocr('2405-0000000032.pdf', './pdf_downloads/2405-0000000032.pdf')

In [ ]:
result.json()

In [ ]:
from IPython.display import Markdown, display

In [ ]:
display(Markdown(result.json()[0]['natural_text'])) # texts

# OCR from downloaded pdf

In [ ]:
pdf_files = glob.glob('./pdf_downloads/*.pdf')
pdf_files

# Convert pdf to image

In [ ]:
from pdf2image import convert_from_path

In [ ]:
def pdf_to_images(pdf_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Convert PDF to list of images
    images = convert_from_path(pdf_path)
    
    # Save each image
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f'page_{i+1}.png')
        image.save(image_path, 'PNG')

In [ ]:
output_path = './pdf_images'
os.makedirs(output_path, exist_ok=True)

In [ ]:
pdf_files

In [ ]:
pdf_image_path_list = []
for file in tqdm(pdf_files):
    pdf_image_path = output_path+'/'+file.split('/')[-1].split('.pdf')[0]
    pdf_image_path_list.append(pdf_image_path)
    os.makedirs(pdf_image_path, exist_ok=True)
    output_dir = pdf_image_path  # Output folder for images
    pdf_to_images(file, output_dir)
    # break

In [ ]:
pdf_image_path_list

## OCR image

In [ ]:
def image_ocr(img, img_path):

    url = "x"

    payload = {'stringInput': 'ocr'}
    files=[
    ('file',(img, open(img_path,'rb'),'image/jpeg'))
    ]
    headers = {
    'accept': 'image/jpeg'
    }

    response = requests.request("POST", url, headers=headers, data=payload, files=files)

    return response.json()

In [ ]:
datas = {
        'source' : [],
        'page' : [],
        'result' : []
}
for file in pdf_image_path_list:
    # ocr_text = []
    # pdf_page = []
    # source = []

    pages = glob.glob(file+'/*')
    pages = sorted(pages, key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
    if len(pages) > 5: # skip cause took too long
        continue
    # print(pages)
    for idx, p in tqdm(enumerate(pages)):
        # pdf_page.append(idx+1)
        # source.append(p.split('/')[-2])
        datas['source'].append(p.split('/')[-2])
        datas['page'].append(idx+1)
        # print(idx, p)
        datas['result'] += image_ocr(p.split('/')[-1], p)
        # datas['ocr_text'] += ocr_text)
    # datas['source'].append(source)
    # datas['page'].append(idx+1)
    # datas['ocr_text'].append(ocr_text)
    # break

In [ ]:
datas

## save to data frame

In [ ]:
df = pd.DataFrame.from_dict(datas)

In [ ]:
df

In [ ]:
df['ocr_text'] = df['result'].apply(lambda x : x['natural_text'])

In [ ]:
df